In [1]:
import sys
sys.path.append("..")
from utils.download_dataset import download_dataset, extract_zip

In [2]:
import numpy as np
import PIL

from fastai.vision.all import *
from pathlib import Path
from tqdm.auto import tqdm
from shutil import copyfile, rmtree

In [3]:
DATA_PATH = Path.cwd()/'data'
if not DATA_PATH.exists():
    DATA_PATH.mkdir(exist_ok=True)

fpath = download_dataset(dataset_name='FISH', dest_dir=DATA_PATH)
fpath

File exists: /home/ubuntu/github/cv_fastai/segmentation/data/FISH.zip


Path('/home/ubuntu/github/cv_fastai/segmentation/data/FISH.zip')

In [4]:
dir_path = extract_zip(fpath)
dir_path

Directory exists: /home/ubuntu/github/cv_fastai/segmentation/data/FISH


Path('/home/ubuntu/github/cv_fastai/segmentation/data/FISH')

In [5]:
fish_classes = []
for ii in (dir_path/'Fish_Dataset').ls():
    if ii.is_dir():
        fish_classes.append(ii.stem)
fish_classes

['Sea Bass',
 'Trout',
 'Hourse Mackerel',
 'Shrimp',
 'Red Mullet',
 'Black Sea Sprat',
 'Red Sea Bream',
 'Striped Red Mullet',
 'Gilt-Head Bream']

In [6]:
fish_paths = [get_files(dir_path/'Fish_Dataset'/c/c, extensions='.png') for c in fish_classes] 
img_fpaths = [img_path for fish_path in fish_paths for img_path in fish_path]
len(img_fpaths)

9000

In [7]:
msk_fpaths = [Path(str(img_path.parent) + ' GT')/img_path.name for img_path in img_fpaths]

print(len(msk_fpaths))
print(np.sum([ii.exists() for ii in msk_fpaths]))

9000
9000


In [14]:
path_images = dir_path/'images'
if not path_images.exists():
    path_images.mkdir(exist_ok=True)
    
path_labels = dir_path/'labels'
if not path_labels.exists():
    path_labels.mkdir(exist_ok=True)

In [28]:
for img_fpath, msk_fpath in tqdm(zip(img_fpaths, msk_fpaths)):
    copyfile(img_fpath, path_images/f'{img_fpath.parent.stem}_{img_fpath.name}')
    msk = Image.open(msk_fpath)
    msk = np.array(msk).astype(np.uint8)
    msk_img = Image.fromarray(msk)
    msk_img.save(path_labels/f'{img_fpath.parent.stem}_{img_fpath.name}')
    #copyfile(msk_fpath, path_labels/f'{img_fpath.parent.stem}_{img_fpath.name}')

0it [00:00, ?it/s]

In [29]:
new_img_fpaths = get_files(path_images)
new_msk_fpaths = get_files(path_labels)
assert len(new_img_fpaths) == len(img_fpaths)
assert len(new_msk_fpaths) == len(img_fpaths)